In [1]:
using Random
using JuMP
using TimerOutputs

In [30]:
function p_median(to, num_facilities, num_customers, num_locations)
    @timeit to "constants" begin
        Random.seed!(10)
        customer_locations = [rand(1:num_locations) for _ in 1:num_customers]
    end

    @timeit to "model" begin
        model = Model()
    end
    @timeit to "variable" begin
        @timeit to "facility" begin
            has_facility = @variable(model, [1:num_locations], Bin)
        end
        @timeit to "closest" begin
            is_closest = @variable(model, [1:num_locations, 1:num_customers], Bin)
        end
    end

    @timeit to "objective" begin
        @objective(model, Min,
        sum(abs(customer_locations[customer] - location)
            * is_closest[location, customer]
            for customer in 1:num_customers, location in 1:num_locations))
    end

    @timeit to "constraint" begin
        @timeit to "customer loop" begin
            for customer in 1:num_customers
                @timeit to "facility" begin
                    # `location` can't be closest for `customer` if there is no facility.
                    @constraint(model,
                        [location in 1:num_locations],
                        is_closest[location, customer] <= has_facility[location])
                end
                @timeit to "closest" begin
                    # One facility must be the closest for `customer`.
                    @constraint(model,
                        sum(is_closest[location, customer]
                            for location in 1:num_locations) == 1)
                end
            end
        end

        @timeit to "sum facility" begin
            # Must place all facilities.
            @constraint(model, sum(has_facility) == num_facilities)
        end
    end
    
    return
end

p_median (generic function with 1 method)

In [38]:
to = TimerOutput()
@timeit to "P median" begin
    p_median(to, 100, 100, 5000);
end
to

 ────────────────────────────────────────────────────────────────────────────
                                     Time                   Allocations      
                             ──────────────────────   ───────────────────────
      Tot / % measured:           2.22s / 99.5%           1.01GiB / 100%     

 Section             ncalls     time   %tot     avg     alloc   %tot      avg
 ────────────────────────────────────────────────────────────────────────────
 P median                 1    2.21s   100%   2.21s   1.01GiB  100%   1.01GiB
   constraint             1    1.61s  73.0%   1.61s    898MiB  86.8%   898MiB
     customer loop        1    1.61s  72.9%   1.61s    897MiB  86.7%   897MiB
       facility         100    1.44s  65.4%  14.4ms    848MiB  82.0%  8.48MiB
       closest          100    166ms  7.53%  1.66ms   48.9MiB  4.73%   501KiB
     sum facility         1    658μs  0.03%   658μs    870KiB  0.08%   870KiB
   variable               1    402ms  18.2%   402ms    104MiB  

In [32]:
function cont5(to, n)
    @timeit to "constants" begin
        m = n
        n1 = n-1
        m1 = m-1
        dx = 1/n
        T = 1.58
        dt = T/m
        h2 = dx^2
        a = 0.001
        yt = [0.5*(1 - (j*dx)^2) for j in 0:n]
    end

    @timeit to "model" begin
        model = Model()
    end
    @timeit to "variable" begin
        @timeit to "y" begin
            y = @variable(model, [0:m,0:n], lower_bound=0, upper_bound=1)
        end
        @timeit to "u" begin
            u = @variable(model, [1:m], lower_bound=-1, upper_bound=1)
        end
    end

    @timeit to "objective" begin
        @objective(model, Min, y[0,0])
    end

    @timeit to "constraint" begin
        @timeit to "PDE" begin
            for i in 0:m1
                for j in 1:n1
                    @constraint(model,
                        h2*(y[i+1,j] - y[i,j])
                        == 0.5*dt*(y[i,j-1] - 2*y[i,j] +
                            y[i,j+1] + y[i+1,j-1] - 2*y[i+1,j] + y[i+1,j+1]) )
                end
            end
        end

        @timeit to "Initial conditions" begin
            for j in 0:n
                @constraint(model, y[0,j] == 0)
            end
        end

        @timeit to "Boundary conditions" begin
            for i in 1:m
                @constraint(model,
                    y[i,2]   - 4*y[i,1]  + 3*y[i,0] == 0)
                @constraint(model,
                    y[i,n-2] - 4*y[i,n1] + 3*y[i,n]== (2*dx)*(u[i] - y[i,n]))
            end
        end
    end
    
    return
end

cont5 (generic function with 2 methods)

In [35]:
to = TimerOutput()
@timeit to "cont5" begin
    cont5(to, 500);
end
to

 ──────────────────────────────────────────────────────────────────────────────
                                       Time                   Allocations      
                               ──────────────────────   ───────────────────────
       Tot / % measured:            1.18s / 98.7%            452MiB / 100%     

 Section               ncalls     time   %tot     avg     alloc   %tot      avg
 ──────────────────────────────────────────────────────────────────────────────
 cont5                      1    1.17s   100%   1.17s    452MiB  100%    452MiB
   constraint               1    923ms  79.1%   923ms    360MiB  79.6%   360MiB
     PDE                    1    921ms  78.9%   921ms    357MiB  79.0%   357MiB
     Boundary condi...      1   1.88ms  0.16%  1.88ms   2.04MiB  0.45%  2.04MiB
     Initial condit...      1    459μs  0.04%   459μs    540KiB  0.12%   540KiB
   variable                 1    244ms  20.9%   244ms   92.3MiB  20.4%  92.3MiB
     y                      1    244ms 